In [56]:
!/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip
!pip install tensorflow
!pip install keras

Requirement already up-to-date: pip in /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages (20.1.1)


In [69]:
import numpy as np

import os
import cv2
import glob

from tqdm.notebook import tqdm as log_progres

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import load_model, save_model
from keras.layers import Input, GlobalAveragePooling2D, GlobalMaxPooling2D
import keras.backend as K
from keras.models import Model, load_model
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.utils import to_categorical

In [70]:
train_dir = os.path.join('/GoogleLandmarkRetrieval2020', 'train')
index_dir = os.path.join('/GoogleLandmarkRetrieval2020', 'index')
test_dir = os.path.join('/GoogleLandmarkRetrieval2020', 'test')

In [71]:
print(train_dir)

/GoogleLandmarkRetrieval2020/train


In [72]:
batch_size = 20
epochs = 1 
IMG_HEIGHT = 28
IMG_WIDTH = 28

In [73]:
from keras.datasets import mnist
(xt, yt), (xte, yte) = mnist.load_data()

print(xt)

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


In [74]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [75]:
train = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory='train/0/0',
                                                           shuffle=False,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical')

Found 6120 images belonging to 16 classes.


In [76]:
val = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory='test',
                                                              shuffle=False,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='categorical')


Found 1129 images belonging to 16 classes.


In [77]:
print(val)

In [80]:
train1, _ = next(train)
val1, _ = next(val)
train1 = train1.reshape(60, 784)
val1 = val1.reshape(60, 784)
print(train1)

[[0.9607844  0.9725491  0.9921569  ... 0.24313727 0.2784314  0.28235295]
 [0.32156864 0.32941177 0.43529415 ... 0.47058827 0.46274513 0.4039216 ]
 [0.3921569  0.32156864 0.3254902  ... 0.24313727 0.2901961  0.2509804 ]
 ...
 [0.28627452 0.45882356 0.7137255  ... 0.6745098  0.86274517 0.5372549 ]
 [0.6784314  0.8745099  0.5411765  ... 0.1764706  0.4156863  0.4431373 ]
 [0.2392157  0.42352945 0.4431373  ... 0.3254902  0.35686275 0.17254902]]


In [81]:
# include top should be False to remove the softmax layer
pretrained_model = VGG16(include_top=False, weights='imagenet')
pretrained_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [82]:
vgg_features_train = pretrained_model.predict(train)
vgg_features_val = pretrained_model.predict(val)

In [ ]:
# OHE target column
train_target = to_categorical(train.labels)
val_target = to_categorical(val.labels)

In [ ]:

#input_shape=(7,7,512)

from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout

model2 = Sequential()
#model2.add(Flatten(input_shape=(20,4608,1)))
model2.add(Dense(100, input_shape = (6120, 1), activation='relu'))
#model2.add(Dropout(0.5))
#model2.add(BatchNormalization())
model2.add(Dense(10, activation='relu'))

# compile the model
model2.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

model2.summary()

# train model using features generated from VGG16 model
model2.fit(vgg_features_train, train_target, epochs=1, batch_size=20, validation_data=(vgg_features_val, val_target))